In [1]:
grobid_server = "https://kermitt2-grobid.hf.space"

In [2]:
!pip install requests


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import requests

def check_grobid_server():
    url = f"{grobid_server}/api/isalive"
    
    try:
        # Send a GET request to the URL
        # 
        print(f"checking {url}...")
        response = requests.get(url)
        print("checked!")

        # Check if the status code indicates success (200-299)
        if response.status_code == 200 and response.text == "true":
            print(f"Server is up and running at {url}.")
            return True
        else:
            print(f"Server responded with status code: {response.status_code}")
            return False
    
    except requests.ConnectionError:
        print(f"Unable to connect to {url}. The server may be down or unreachable.")
        return False
    
# Test the server
assert check_grobid_server()

checking https://kermitt2-grobid.hf.space/api/isalive...
checked!
Server is up and running at https://kermitt2-grobid.hf.space/api/isalive.


In [8]:
input_path = "C:/Users/anton/OneDrive/Área de Trabalho/extracao/data/raw"
output_path = "..\data/interim"  # Replace with the desired output folder path

import os


# Make sure the output folder exists
if not os.path.exists(output_path):
    os.makedirs(output_path)

# List all files in the folder
files = []

for filename in os.listdir(input_path):
    file_path = os.path.join(input_path, filename)
    if os.path.isfile(file_path):
        # Add the file and its size to the list
        files.append((file_path, os.path.getsize(file_path)))

print(f"I have {len(files)} files to process!")


I have 21 files to process!


In [9]:
!pip install tqdm

     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 217.9 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.7 kB 217.9 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.7 kB 327.7 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.7 kB 327.7 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.7 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 189.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.5 kB ? eta -:--:--
   ------------------------------------ --- 71.7/78.5 kB ? eta -:--:--
   ---------------------------------------- 78.5/78.5 kB 726.6 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\anton\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [10]:
import subprocess
import time
import random
import os
from tqdm import tqdm 

# Sort files by size (smallest first)
files.sort(key=lambda x: x[1])
random.shuffle(files)

# Process each file in order of size com tqdm
for file_path, file_size in tqdm(files, desc="Processando arquivos", unit="file"):
    # Skip processing if the file size is zero
    if file_size == 0:
        tqdm.write(f"Skipped {file_path} (file size is zero)")
        continue

    # Extract the filename from the path
    filename = os.path.basename(file_path)

    # Prepare the output filename
    output_filename = f"{filename}.grobid.tei.xml"
    output_file_path = os.path.join(output_path, output_filename)

    # Check if the output file exists and its size is greater than 0
    if not os.path.exists(output_file_path) or os.path.getsize(output_file_path) == 0:
        # Prepare the curl command
        tqdm.write(f"Trying to extract {file_path} from PDF to TEI...")

           
        curl_command = [
            "curl", "-v", 
            "-H", "Accept: application/xml", 
            "--form", "consolidateCitations=1", 
            "--form", "includeRawCitations=1", 
            "--form", "segmentSentences=1",
            "--form", "generateIDs=1", 
            "--form", f"input=@{file_path}", 
            f"{grobid_server}/api/processFulltextDocument",
        ]

        # Open the output file to redirect the curl output into it
        with open(output_file_path, "w") as output_file:
            try:
                # Run the process with a timeout of 10 minutes (600 seconds)
                result = subprocess.run(curl_command, stdout=output_file, stderr=subprocess.PIPE, timeout=600)
                #if result.stderr:
                #    tqdm.write("STDERR:")
                #    tqdm.write(result.stderr.decode())

            except subprocess.TimeoutExpired:
                tqdm.write("The process did not finish within 10 minutes and was terminated.")

        # Check if the output file size is zero
        if os.path.getsize(output_file_path) == 0:
            tqdm.write(f"{output_file_path} is empty. NOT PROCESSED.")
        else:
            tqdm.write(f"Processed {filename} -> {output_file_path}")
            time.sleep(20)
    else:
        tqdm.write(f"Skipped {filename} (output file already exists and is non-empty)")

Processando arquivos:   0%|          | 0/21 [00:00<?, ?file/s]

Trying to extract C:/Users/anton/OneDrive/Área de Trabalho/extracao/data/raw\Austrian-persistence-Capitalbased-business-cycle-theory-and-the-dynamics-of-investment-spending_2006_.pdf from PDF to TEI...


Processando arquivos:   0%|          | 0/21 [01:40<?, ?file/s]

Processed Austrian-persistence-Capitalbased-business-cycle-theory-and-the-dynamics-of-investment-spending_2006_.pdf -> ..\data/interim\Austrian-persistence-Capitalbased-business-cycle-theory-and-the-dynamics-of-investment-spending_2006_.pdf.grobid.tei.xml


Processando arquivos:   5%|▍         | 1/21 [02:00<40:13, 120.68s/file]

...

Processando arquivos:  95%|█████████▌| 20/21 [30:23<01:00, 60.57s/file]

Trying to extract C:/Users/anton/OneDrive/Área de Trabalho/extracao/data/raw\Editorial-Private-is-profit-and-the-public-is-dead_2018_Routledge-infotandfcouk.pdf from PDF to TEI...


Processando arquivos:  95%|█████████▌| 20/21 [30:28<01:00, 60.57s/file]

Processed Editorial-Private-is-profit-and-the-public-is-dead_2018_Routledge-infotandfcouk.pdf -> ..\data/interim\Editorial-Private-is-profit-and-the-public-is-dead_2018_Routledge-infotandfcouk.pdf.grobid.tei.xml


Processando arquivos: 100%|██████████| 21/21 [30:48<00:00, 88.04s/file]


In [11]:
import re

text = "Data science is cool as you get to work with real-world data"
matches = re.findall(r"data", text)
print(matches) 

['data']


In [12]:
matches = re.findall(r"data", text, re.IGNORECASE)
print(matches)  

['Data', 'data']


In [13]:
texto = "Ligue para (11) 98765-4321 ou (21) 3456-7890"
telefones = re.findall(r"\(\d{2}\)\s?\d{4,5}-\d{4}", texto)
print(telefones) 

['(11) 98765-4321', '(21) 3456-7890']


In [14]:
texto = "Ligue para (11) 98765-4321 ou (21) 3456-7890, ou 99215-0859"
telefones = re.findall(r"\(\d{2}\)\s?\d{4,5}-\d{4}", texto)
telefones = re.findall(r"(?:\(\d{2}\)\s?)?\d{4,5}-\d{4}", texto)

print(telefones) 

['(11) 98765-4321', '(21) 3456-7890', '99215-0859']


In [15]:
texto = "Estes sao meus email: pedro.calais@gmail.com e pcalais@dcc.ufmg.br"
textos = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", texto)
print(textos)

['pedro.calais@gmail.com', 'pcalais@dcc.ufmg.br']
